In [92]:
import json
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import nltk


In [93]:
import json
with open('data.json') as json_data:
    intents = json.load(json_data)
    

In [94]:
lines = []
with open('./train_sentences/train_emotion.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        str,emo = line.split(';')
        if 'sadness' in emo: 
                if str not in intents['intents'][3]['patterns'] and len(intents['intents'][3]['patterns']) <= 400:
                        intents['intents'][3]['patterns'].append(str)
        if 'joy' in emo: 
                if str not in intents['intents'][4]['patterns']and len(intents['intents'][4]['patterns']) <= 400:
                        intents['intents'][4]['patterns'].append(str)
        if 'anger' in emo: 
                if str not in intents['intents'][5]['patterns']and len(intents['intents'][5]['patterns']) <= 400:
                        intents['intents'][5]['patterns'].append(str)
        if 'fear' in emo: 
                if str not in intents['intents'][6]['patterns']and len(intents['intents'][6]['patterns']) <= 400:
                        intents['intents'][6]['patterns'].append(str)


In [95]:
lines = []
with open('./train_sentences/train_greet.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][0]['patterns']:
            intents['intents'][0]['patterns'].append(line)

lines = []
with open('./train_sentences/train_bye.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][1]['patterns']:
            intents['intents'][1]['patterns'].append(line)

lines = []
with open('./train_sentences/train_thanks.txt') as train_data:
        lines = train_data.readlines()

for line in lines:
        if line not in intents['intents'][2]['patterns']:
            intents['intents'][2]['patterns'].append(line)


with open('data2.json', 'w') as outfile:
        json.dump(intents, outfile)


In [96]:
with open('data2.json') as json_data:
    intents = json.load(json_data)

In [97]:
training_sentences = []
training_labels = []
labels = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

num_classes = len(labels)

In [98]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

training_labels

array([3, 3, 3, ..., 1, 1, 1])

In [99]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<00V>"

#import spacy 
#emb = spacy.load('en_core_web_md')

# sequences = []
# for sentence in training_sentences:
#     word_vectors = emb(sentence)
#     tmp = []
#     for word in word_vectors:
#         tmp.append(word.vector.tolist())
#     sequences.append(tmp)

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen = max_len)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, training_labels, test_size = 0.2, random_state = 42)

In [100]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length = max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16,activation = 'relu'))
model.add(Dense(16,activation = 'relu'))
model.add(Dense(num_classes, activation = 'softmax'))

model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d_11  (None, 16)               0         
  (GlobalAveragePooling1D)                                       
                                                                 
 dense_36 (Dense)            (None, 16)                272       
                                                                 
 dense_37 (Dense)            (None, 16)                272       
                                                                 
 dense_38 (Dense)            (None, 7)                 119       
                                                                 
Total params: 16,663
Trainable params: 16,663
Non-trainable params: 0
_________________________________________________

In [101]:
history = model.fit(X_train, np.array(y_train), epochs = 500)

Epoch 1/500
45/45 [==============================] - 0s 965us/step - loss: 1.8907 - accuracy: 0.2338
Epoch 2/500
45/45 [==============================] - 0s 727us/step - loss: 1.7387 - accuracy: 0.2240
Epoch 3/500
45/45 [==============================] - 0s 1ms/step - loss: 1.6029 - accuracy: 0.2212
Epoch 4/500
45/45 [==============================] - 0s 1ms/step - loss: 1.5173 - accuracy: 0.2479
Epoch 5/500
45/45 [==============================] - 0s 1ms/step - loss: 1.4752 - accuracy: 0.2942
Epoch 6/500
45/45 [==============================] - 0s 1ms/step - loss: 1.4391 - accuracy: 0.3961
Epoch 7/500
45/45 [==============================] - 0s 1ms/step - loss: 1.4025 - accuracy: 0.4080
Epoch 8/500
45/45 [==============================] - 0s 1ms/step - loss: 1.3635 - accuracy: 0.4143
Epoch 9/500
45/45 [==============================] - 0s 1ms/step - loss: 1.3175 - accuracy: 0.5169
Epoch 10/500
45/45 [==============================] - 0s 957us/step - loss: 1.2684 - accuracy: 0.5119
Epo

In [102]:
from tensorflow.keras import metrics
model.evaluate(X_test, y_test)

12/12 [==============================] - 0s 1ms/step - loss: 7.9469 - accuracy: 0.5966


[7.946899890899658, 0.5966386795043945]

In [103]:
model.save("chat_model")

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets
